# Code to Scrap Data from Indeed.com

In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import cloudscraper
import pandas as pd

In [2]:
def get_url(position, location):
    template = 'https://in.indeed.com/jobs?q={}&l={}'
    url = template.format(position, location)
    return url

In [3]:
def main(pos, loc):

    position = []
    company_name = []
    location = []
    salary = []
    description = []

    url = get_url(pos, loc)  # create the url while passing in the position and location.
    scraper = cloudscraper.create_scraper() 
    
    while True:
        response = scraper.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        print(url)
        
        temp = soup.find_all('div', 'css-1m4cuuf e37uo190')
        for i in temp:
            position.append(i.text.strip())
        
        temp = soup.find_all('span','companyName')
        for i in temp:
            company_name.append(i.text.strip())
            
        temp = soup.find_all('div','companyLocation')
        for i in temp:
            location.append(i.text.strip())
        
        temp = soup.find_all('div','heading6 tapItem-gutter metadataContainer noJEMChips salaryOnly')
        for i in temp:
            salary.append(i.text)
        
        temp = soup.find_all('div','job-snippet')
        for i in temp:
            description.append(i.text)

            
        try:
            url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
            print(url)
            delay = randint(1, 10)
            sleep(delay)
        except AttributeError:
            break

    d = {'Position':position, 'Company':company_name, 'Location':location, 'Description':description}
    df = pd.DataFrame(d)
    print(df)

In [4]:
main('python+developer', 'Mumbai')

https://in.indeed.com/jobs?q=python+developer&l=Mumbai
                                             Position  \
0                                 Python Developernew   
1                                 Python Developernew   
2                                 Python Developernew   
3                           Python Software Developer   
4                                    Python Developer   
5                                   C++ Programmernew   
6                                 Python Developernew   
7                   Application Developer - Pythonnew   
8                                    Python Developer   
9   Python Programming Language Application Develo...   
10                                   Python Developer   
11                                   Python Developer   
12                                   Python Developer   
13                                   Python Developer   
14                                   Python Developer   

                        Company 